In [ ]:
!pip install ucimlrepo pandas numpy scikit-learn

In [ ]:
# ------- Import Dataset -------------------

from ucimlrepo import fetch_ucirepo

# fetch dataset
air_quality = fetch_ucirepo(id=360)

# data (as pandas dataframes)
X = air_quality.data.features
y = air_quality.data.targets # unused?

In [ ]:
# Combine Date and Time into a single DateTime column -> only 14 features
import pandas as pd
import numpy as np

X['DateTime'] = pd.to_datetime(X['Date'] + ' ' + X['Time']) # Format (JJJJ-MM-DD HH:MM:SS, e.g.: 2004-03-10 18:00:00)
X = X.set_index('DateTime')
X.drop(['Date', 'Time'], axis=1, inplace=True)  # Remove the original Date and Time columns
X.replace(-200, np.nan, inplace=True)
X.replace(-200.0, np.nan, inplace=True)

In [100]:
pd.set_option('display.width', 1000)
print(X.head())

                     CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)     T    RH      AH
DateTime                                                                                                                                                  
2004-03-10 18:00:00     2.6       1360.0     150.0      11.9         1046.0    166.0        1056.0    113.0        1692.0       1268.0  13.6  48.9  0.7578
2004-03-10 19:00:00     2.0       1292.0     112.0       9.4          955.0    103.0        1174.0     92.0        1559.0        972.0  13.3  47.7  0.7255
2004-03-10 20:00:00     2.2       1402.0      88.0       9.0          939.0    131.0        1140.0    114.0        1555.0       1074.0  11.9  54.0  0.7502
2004-03-10 21:00:00     2.2       1376.0      80.0       9.2          948.0    172.0        1092.0    122.0        1584.0       1203.0  11.0  60.0  0.7867
2004-03-10 22:00:00     1.6       1272.0      51.0       6.5          

In [101]:
# Invert PT08.S3(NOx)
X['PT08.S3(NOx)'] = -1 * X['PT08.S3(NOx)']  # Reversing the sign

In [102]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Initialize the Min-Max Scaler
scaler = MinMaxScaler()

# Select only the columns to be scaled
features_to_scale = X.columns  # As 'DateTime' is an index, it won't be included

# Apply the scaler to the features
X_scaled = pd.DataFrame(scaler.fit_transform(X[features_to_scale]), columns=features_to_scale, index=X.index)

# Now X_scaled contains the normalized data, with 'DateTime' as the index

In [103]:
#changes nothing for linear regression -> forward fill NA values
X_scaled.fillna(method='ffill', inplace=True)

C:\Users\Tobias\AppData\Local\Temp\ipykernel_356\3026123231.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X_scaled.fillna(method='ffill', inplace=True)


In [ ]:
# Linear Regression
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Assuming df is your DataFrame
# Set your target feature and predictors
target = 'CO(GT)'  # Replace with your target feature's name

predictors = X_scaled.columns[X_scaled.columns != target]

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled[predictors], X_scaled[target], test_size=0.2, random_state=42)

# Initialize and train the model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")